# YAML Files
---
These files are used to configure and organize the website's contents.

In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
#Set the book.
book='quant'

In [24]:
# Always run this before any of the following cells
import pandas as pd
import numpy as np
import csv
import logging
import subprocess
import yaml
import builder as bd
from pathlib import Path
base_path=Path('..')
book_path= base_path / book

In [25]:
# Configuration
config = bd.load_yaml_file(book_path / '_config.yml') # Load the file.
toc = bd.load_yaml_file(book_path / '_toc.yml') # Load the file.
config_xl= pd.read_excel('../book.xlsx', sheet_name = '_config_yml', header=None, index_col=None)
schedule= pd.read_excel('../book.xlsx', sheet_name = 'Schedule',  index_col=None)
content={}
content['Before Class']= pd.read_excel('../book.xlsx', sheet_name = 'Before',  index_col=None)
content['In Class']= pd.read_excel('../book.xlsx', sheet_name = 'During',  index_col=None)
content['Assignment']= pd.read_excel('../book.xlsx', sheet_name = 'Assignments',  index_col=None)


In [26]:
#Prepare 
for index, row in content['Assignment'].iterrows():
    if row['Type']=='Notebook':
        content['Assignment'].loc[index,'Assignment']=row['Assignment']+" [Starter]("+row['Location']+")"
    elif row['Type']=='File':
        content['Assignment'].loc[index,'Assignment']=row['Assignment']+" [Starter]("+config['repository']['url']+"/"+row['Location']+")"
schedule=schedule.merge(content['Assignment'], left_on='Session', right_on='Session', how='left')
#schedule['Due']=schedule['Date'].apply(lambda x: x+ pd.Timedelta(7, unit='D'))

In [27]:
schedule.head()

,Week,Session,Date,Day,Topic,Summary,Publish,Assignment,Location,Due,Type
0,1,1,2020-08-31,Mon,Welcome and class overview,In this class we will simply be providing a hi...,1,"In this starter assignment, we are just going ...",files/assignments/01starter.xlsx,2020-09-07,File
1,1,2,2020-09-03,Thu,Intro to Management Science,More detail on specific approches.,1,This will test your ability to use Jupyter not...,assignments/02starter,2020-09-10,Notebook
2,2,NaN,2020-09-07,Mon,Labor day – no class,NaN,0,NaN,NaN,NaT,NaN
3,2,3,2020-09-08,Tue,Linear Programming 1,TBD,1,NaN,NaN,NaT,NaN
4,2,4,2020-09-10,Thu,Linear Programming 2,TBD,1,NaN,NaN,NaT,NaN


In [28]:
schedule_str=bd.pandas_to_md(schedule, book_path / 'schedule.md', 'Schedule', \
                             include = ['Week', 'Session', 'Date', 'Day', 'Topic', 'Summary', 'Assignment', 'Due'])

Converting datetime to 
Converting datetime to 
Outputting file: ../quant/schedule.md


In [29]:
#Generate Session Files
toc=bd.generate_sessions(config, toc, 2, schedule_str, book_path / 'sessions',content, ['Before Class', 'In Class', 'Assignment'])

Outputting  session1.md
Outputting  session2.md
Outputting  session3.md
Outputting  session4.md


In [30]:
bd.update_yaml_file(book_path / '_toc.yml', toc)

Updating the file: ../quant/_toc.yml
